In [1]:
import torch

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModel
from transformers import LogitsProcessor
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

device = 'cuda'
model_id = 'gpt2-xl'
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id, pad_token_id=tokenizer.eos_token_id).to(device)

# Examples from the Readme

In [ ]:
import clownfish
from clownfish import parser_for_type, create, create_api
import importlib
importlib.reload(clownfish)

In [11]:
from pydantic import BaseModel

class Ingredient(BaseModel):
    type: str
    count: float
    
class ShoppingList(BaseModel):
    list: list[Ingredient]

create(tokenizer, model, 'cuda', ShoppingList, "A shopping list that has 3 eggs, 7 apples and 5 loaves of bread in JSON: ")


  { "list": [ { "type": "egg", "count":3 }, { "type": "apple", "count":7 }, { "type": "bread", "count":5 } ] }

(ShoppingList(list=[Ingredient(type='egg', count=3.0), Ingredient(type='apple', count=7.0), Ingredient(type='bread', count=5.0)]),
 tensor(1.8488))

In [13]:
class Address(BaseModel):
    street_number: float
    street_name: str
    zip_code: str
    
ONE_SHOT = """The following address parsed as JSON: 111 Central Park N, New York, NY { "street_number": 111, "street_name": "Central Park N", zip_code: "10026" }
The following address parsed as JSON: """

print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "111 Central Park N, New York, NY 10026 "))
print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "123 Main St, New York, NY 10022 "))
print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "I am a banana "))
print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "1188 Mission St, Apt 17, San Francisco, CA 94017 "))
print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "12 and a half 1st St, Chicago, IL, 39443 "))
print(create(tokenizer, model, 'cuda', Address, ONE_SHOT + "Chicago, IL "))

  { "street_number":111, "street_name": "Central Park N", "zip_code": "10026" }(Address(street_number=111.0, street_name='Central Park N', zip_code='10026'), tensor(1.5181))
  { "street_number":123, "street_name": "Main St", "zip_code": "10022" }(Address(street_number=123.0, street_name='Main St', zip_code='10022'), tensor(1.3475))
  { "street_number":111, "street_name": "I am a banana", "zip_code": "10026" }(Address(street_number=111.0, street_name='I am a banana', zip_code='10026'), tensor(1.6365))
  { "street_number":1188, "street_name": "Mission St", "zip_code": "94017" }(Address(street_number=1188.0, street_name='Mission St', zip_code='94017'), tensor(1.4449))
  { "street_number":12, "street_name": "1st St", "zip_code": "60625" }(Address(street_number=12.0, street_name='1st St', zip_code='60625'), tensor(1.6610))
  { "street_number":2, "street_name": "Chicago", "zip_code": "60606" }(Address(street_number=2.0, street_name='Chicago', zip_code='60606'), tensor(1.9208))


In [ ]:
import openai
openai.api_key = input()

In [64]:
class Ingredient(BaseModel):
    type: str
    count: float
    
class ShoppingList(BaseModel):
    list: list[Ingredient]

create_api(tokenizer, ShoppingList, "A shopping list that has 3 eggs, 7 apples and 5 loaves of bread in JSON: ", 1000)

{ "list": [{"type":"eggs", "count":3 },{ "type":"apples", "count":7 },{ "type":"bread", "count":5 }] }

Complete with usage: 360 , prompt + final token count 60


ShoppingList(list=[Ingredient(type='eggs', count=3.0), Ingredient(type='apples', count=7.0), Ingredient(type='bread', count=5.0)])

In [43]:
create_api(tokenizer, ShoppingList, "A shopping list in the format { \"list\": [{\"type\":str, \"count\": int}, ...] } that has 3 eggs, 7 apples and 5 loaves of bread in JSON: ", 1000)

{ "list": [{"type":"eggs", "count":3}, {"type":"apples", "count":7}, {"type":"loaves of bread", "count":5}] }

Complete with usage: 312 , prompt + final token count 82


ShoppingList(list=[Ingredient(type='eggs', count=3.0), Ingredient(type='apples', count=7.0), Ingredient(type='loaves of bread', count=5.0)])

# A GPT2 Is easily distracted (and doesn't know history)
What was that about party again?

Meanwhile, Clownfish will happily oblige

In [14]:
inputs = tokenizer.encode("A list in JSON of the first 3 US presidents with format [{ \"number\": int, \"name\": string, \"party\": \"dem\" or \"rep\"}]: [ { \"name\":", return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=100)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
A list in JSON of the first 3 US presidents with format [{ "number": int, "name": string, "party": "dem" or "rep"}]: [ { "name": "George Washington", "number": 1 }, { "name": "James Madison", "number": 2 }, { "name": "John Adams", "number": 3 } ]

The following example shows how to use the JSON API to retrieve the first three US presidents.

GET /


In [18]:
class President(BaseModel):
    full_name: str
    party: str
    
class Response(BaseModel):
    list: list[President]
    
create(tokenizer, model, 'cuda', Response, "A list in JSON of the first 3 US presidents: ")


  { "list" : [ { "full_name" : "George Washington", "party" : "Republican" }, { "full_name" : "Thomas Jefferson", "party" : "Democratic" }, { "full_name" : "James Madison", "party" : "Republican" }, { "full_name" : "John Adams", "party" : "Democratic" } ] }

(Response(list=[President(full_name='George Washington', party='Republican'), President(full_name='Thomas Jefferson', party='Democratic'), President(full_name='James Madison', party='Republican'), President(full_name='John Adams', party='Democratic')]),
 tensor(1.8934))

In [66]:
create_api(tokenizer, Response, "A list in JSON of the first 3 US presidents: ", 1000)

{ "list": [{"full_name":"George Washington", "party":"Independent"}, {"full_name":"John Adams", "party":"Federalist"}, {"full_name":"Thomas Jefferson", "party":"Democratic-Republican"}] }

Complete with usage: 183 , prompt + final token count 60


Response(list=[President(full_name='George Washington', party='Independent'), President(full_name='John Adams', party='Federalist'), President(full_name='Thomas Jefferson', party='Democratic-Republican')])

# Should You Trust GPT2 With Your Diet?

Probably not, but it does seem fairly confident you should eat spinahs and not eat lard.

In [21]:
from typing import Literal

class Result(BaseModel):
    healthy: Literal["y", "n"]

ONE_SHOT = """JSON evaluation for the healthiness of chips: { "healthy": "n" }
JSON evaluation for the healthiness of carrots: { "healthy": "y" }
JSON evaluation for the healthiness of """
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "chips: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "lard: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "spinach: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "cake: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "fried donut: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "weofijwef: "))
print(create(tokenizer, model, 'cuda', Result, ONE_SHOT + "salad: "))

  { "healthy":"n" }(Result(healthy='n'), tensor(3.1886))
  { "healthy":"n" }(Result(healthy='n'), tensor(3.0601))
  { "healthy":"y" }(Result(healthy='y'), tensor(3.0842))
  { "healthy":"n" }(Result(healthy='n'), tensor(3.1398))
  { "healthy":"n" }(Result(healthy='n'), tensor(3.2215))
  { "healthy":"y" }(Result(healthy='y'), tensor(3.4784))
  { "healthy":"y" }(Result(healthy='y'), tensor(3.0835))
